<a href="https://colab.research.google.com/github/FabianGD/TeachingAST/blob/master/AST_Homework_nr_2_Integrals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework \#2 -- Scientific computing

In this second part of the programming lessons, we want to build a small Hartree-Fock quantum chemistry program in Python.

## 1 Basis functions


One key element of the Hartree-Fock approach are overlap integrals of the basis functions. We will use Gaussian-type basis function with the form:

$$
\eta_\mu(r) = n_\mu \cdot e^{-\mu r^2 }
$$

<!--For this task we will operate in spherical coordinates, such that in theory, the basis function takes the form:

$$
\eta_\mu(r, \theta, \varphi) = n_\mu \cdot e^{-\mu r^2 }
$$

Of course the function does not have a dependency on $\theta$ and $\varphi$, however, we still need to consider them later on. -->

The parameter $\mu$ is the exponent that defines the basis set. So now, our first task is to find out how to normalise the basis functions, as these Gaussian functions are generally not normalised out-of-the-box. In quantum chemistry, we define normalisation as follows. "$\mathbf{r}$" resembles a three-dimensional space $\mathbb{R}^3$ (such as cartesian or spherical coordinates):

$$
\langle \phi | \phi \rangle = \int \phi^*(\mathbf{r}) \phi(\mathbf{r}) \mathrm{d}\mathbf{r} = 1
$$


### 1.1 Normalise the basis functions (math)

Your first task is to **find the value for the normalisation parameter $n_\mu$**. To do so, solve the following integral for $n_\mu$:

$$
1 = \int\limits_0^{2\pi}  \int\limits_0^{\pi}  \int\limits_0^{\infty} \eta^*_\mu(r) \eta_\mu(r)~r^2 \mathrm{sin}\theta~\mathrm{dr}\,\mathrm{d\theta}\,\mathrm{d\varphi}
$$

Hints: 
- $\int_0^\infty x^2 e^{-ax^2} \mathrm{d}x = \dfrac{\sqrt{\pi}}{4 \cdot a^ {3/2}}\quad \mathrm{for}\quad \Re(a) > 0$.

- The first integral refers to $\varphi$, the second to $\theta$ and the last to $r$.
- The integrals over $\varphi$ and $\theta$ should add up to a factor of $4\pi$.

**Write your answer here.** If you do not know the LaTeX notation, you can also write it as code below. Just double-click on this text cell to edit it.

### 1.2 Calculate the analytical overlap integral (math)

Now that we have the normalised basis functions, we can continue. The second important quantity we need is the overlap between two basis functions. We denote this quantity as $s_{\mu \nu}$. Solve the following integration (the angular components are already integrated):

$$
s_{\mu \nu} = 4  \pi \int\limits_0^{\infty} \eta_\nu(r) \eta_\mu(r)~r^2~\mathrm{dr}
$$

**Write your answer here.** If you do not know the LaTeX notation, you can also write it as code below. Just double-click on this text cell to edit it.

### 1.3 Implementing basis functions in Python





So now we start out building our integrals in Python. To see how the functions compare speed-wise, we will implement three similar approaches to the overlap integral:

**Task**: **Implement the basis function** $\eta_\mu(r)$ in Python.
The normalisation parameter $n_\mu$ should be implemented as a function of $\mu$ (write it `mu` in Python)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Define here the functiion n(mu)
def n(mu):
    norm_factor = ((2 * mu)/ np.pi) ** 0.75
    return norm_factor


# Define here the function eta(mu, r)
def eta(mu, r):
    return n(mu) * np.exp(-mu * r ** 2)


# Now define some basis function and an array of `r` and plot the function
mu = 0.5

r = np.linspace(0, 5, num=1000)
p = eta(mu, r)

plt.plot(r, p)

### 1.4 Build the integration functions (advanced)

Now that we have the basis functions in place, we can think of the integrals (which are basically the most important quantity in a Hartree-Fock program).

1. We will take the naive approach by **calculating the integral via lists** and list comprehension. 
  - Here build a **function of two exponents `mu1` and `mu2` and an array of r values**.

2. Refine the naive approach by **re-implementing the above function using numpy arrays**.

3. Implement the **analytical integral** using your result.
  - This function should only take two exponents `m1` and `m2`!

In the end of the cell you find some code to actually take the time it takes for the function to run. In Colab notebooks, you can use "magic commands" (those with a `%` before that) such as `%timeit`. This will not work in standard Python!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Here goes the numerical integral using lists
def numerical_integ(mu1, mu2, r):
    # here goes your function
    return 4 * np.pi * sum([eta(mu1, ri) * eta(mu2, ri) * ri**2 for ri in r]) * (r[1]-r[0])


# Here goes the numerical integral using numpy
def numerical_integ_numpy(mu1, mu2, r):
  # here goes your function
    return 4 * np.pi * np.sum(eta(mu1, r) * eta(mu2, r) * r**2) * (r[1]-r[0])


# Here goes the analytical integral
def analyt_integ(mu1, mu2):
  # here goes your function
  return n(mu1) * n(mu2) * (np.pi / (mu1 + mu2))**(1.5)


# Define some test values
mu1 = 1.0
mu2 = 6
x = np.linspace(0, 5, num=1000)

# These statements give you a precise timing of the time it takes to run 
%timeit numerical_integ(mu1, mu2, x)
%timeit numerical_integ_numpy(mu1, mu2, x)
%timeit analyt_integ(mu1, mu2)

# Print the results of each integration
print(numerical_integ(mu1, mu2, x))
print(numerical_integ_numpy(mu1, mu2, x))
print(analyt_integ(mu1, mu2))

## 2 Calculating $\pi$ using random numbers

A very peculiar thing to do is calculate the value of $\pi$.
As you might know, $\pi$ is an irrational number and therefore, cannot be easily calculated using fractions. So one approach to calculate $\pi$ is by using its properties as the circle number.

![alt text](https://upload.wikimedia.org/wikipedia/commons/1/1f/Pi_statistisch.png)

The idea is the following: The ratio of the area of a circle relative to the area of a square gives you $\frac{\pi}{4}$:

$$
\dfrac{\mathrm{Area~of~circle}}{\mathrm{Area~of~square}} = \dfrac{\pi \cdot r^2}{(2r)^2} = \dfrac{\pi}{4}
$$

In order to approach the area statistically, we can generate a large amount of random points in the square and then ask Python whether the point is within the circle (see the image, the red dots). The condition is:

$$
x^2 + y^2 \le 1
$$

So, how to go over it:
 - Initiate two variables: 
    1. The total number of points, e.g. `nr_total = 100000`
    1. the number of points within the circle, has to be `nr_circle = 0`
 - Build a `for` loop over the number of random points you want, `nr_total`.
 - Then generate two random numbers for it, one `x` and one `y` value. Use the the `random.random()` function from the `random` library for both. The function generates a random float in the interval $[0, 1.0)$.
 - Then use an `if-else` statement to ask Python whether the point (x,y) is within the circle (with the condition above)
 - If the answer is `True`, then add `1` to the `nr_circle` variable.
 - After the loop, calculate $\pi$ using the formula above. The area of the circle is corresponding to the number of points in the circle.

In [ ]:
import random
import timeit

# Initialise the variables
nr_total = 10
nr_circle = 0

# Build the for loop:
for i in range(nr_total):
    x = random.random()
    y = random.random()

    if x**2 + y**2 <= 1:
        nr_circle = nr_circle + 1

pi = (nr_circle / nr_total) * 4 

print(pi)




## 3 Fill-in-the blanks: Matrix multiplication

In this last task you will build the matrix multiplication function from notebook #2. In order to get a working function, you need to, first, fill in the blanks, indicated by `< ... >` characters and, second, put the parts together in the correct order (and indentation level!).


**The math you may need for it:** Assume we have two matrices $\mathbf{A}$ and $\mathbf{B}$ with elements $a_{ij}$ and $b_{jk}$ (the indices with the same letter *have* to match!), we can multiply them to get a matrix $\mathbf{C}$ with elements $c_{ik}$ like so:

$$
c_{ik} = \sum\limits_j a_{ij} \cdot b_{jk} 
$$

This, of course, has to be conducted *for every entry* of $i$ and $k$.



In [ ]:
# Here goes your function, call it `mat_check()`
def mat_check(mat1, mat2, inout):
  
    # First matrix
    if inout[0] == "row":
        l1 = len(mat1)
    else:
        l1 = len(mat1[0])

    # Second matrix
    if inout[-1] == "row":
        l2 = len(mat2)
    else:
        l2 = len(mat2[0])

    if l1 == l2:
        return True
    else:
        return False


# --> Fill in the blanks! <--

matrix1 = [[3, -6], 
           [-9, 1],
           [ 3, 5]]

matrix2 = [[2, 4, -6], 
           [1, -9, 2]]


# 1. Check the inner and outer indices
if mat_check(matrix1, matrix2, ["col", "row"]):
    
    # print("The dimensions work out!")
    # Here, the code will continue
    pass

else:
    
    raise Exception("Dimensions do not match.")


# 2. Build a new matrix (a matrix of zeros.)
row_1 = len(matrix1)
col_2 = len(matrix2[0])

# Hint: Wherever you do not need a loop variable, in Python you usually assign
# it to an underscore character `_`. (It's just a convention under developers.)
res = [ [0. for _ in range(col_2)] for _ in range(row_1) ]

# 3. Build the computation itself
# Hint: To index an element of the inner list, 
# you can concatenate the indexing --> matrix[1][1]
inner = col_1 = len(matrix1[0])
for i in range(row_1):
    for k in range(col_2):
        res[i][k] = sum([matrix1[i][j] * matrix2[j][k] for j in range(inner)])

print(res)

# --> Now put all the parts together <--
def matrix_matrix(m1, m2):
    # 1. Check the inner and outer indices
    if mat_check(matrix1, matrix2, ["col", "row"]):
        
        print("The dimensions work out!")
        
        # 2. Build a new matrix (a matrix of zeros.)
        row_1 = len(matrix1)
        col_2 = len(matrix2[0])

        # Hint: Wherever you do not need a loop variable, in Python you usually assign
        # it to an underscore character `_`. (It's just a convention under developers.)
        res = [ [0. for _ in range(col_2)] for _ in range(row_1) ]

        # 3. The computation itself 
        inner = col_1 = len(matrix1[0])
        for i in range(row_1):
            for k in range(col_2):
                res[i][k] = sum([matrix1[i][j] * matrix2[j][k] for j in range(inner)])
    
    else:
        raise Exception("Dimensions do not match.")

    return res


# --> Get to trying out <--
# Test your function
mres = matrix_matrix(matrix1, matrix2)

# Test that our calculation is correct using the numpy library (we will get to that later!)
import numpy as np
mres_np = np.matmul(matrix1, matrix2)
print("Is the result correct?", np.all(np.array(mres) == mres_np))